¿Cuál es la categoría con mayor cantidad de reviews promedio en sus aplicaciones? ¿Por qué?

# Cargo el csv y hago ajustes necesarios.

In [ ]:
! pip install pyspark
! pip install -U -q PyDrive
! apt update
! apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,626 B in 3s (1,406 B/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
32 packages can be upgraded. Run 'apt list --upgradable' to see them.
openjdk-8-jdk-headless is already the newest version (

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.sql.types import IntegerType

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({"id" : "1-oPVltZP-8bpOqQtlRLD6yy6TNasoHLP"})
downloaded.GetContentFile("googleplaystore.csv")

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Skipeo una línea con el nombre de la aplicación respectivo porque está mal parseada.

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("googleplaystore.csv", header = True, inferSchema = True)
rdd = df.rdd.filter(lambda x: x.App != "Life Made WI-Fi Touchscreen Photo Frame")

In [ ]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [ ]:
rdd_category = rdd.map(lambda x: (x.Category, 1))
rdd_category.reduceByKey(lambda x, y: x + y).collect()

[('ART_AND_DESIGN', 65),
 ('AUTO_AND_VEHICLES', 85),
 ('BEAUTY', 53),
 ('BOOKS_AND_REFERENCE', 231),
 ('BUSINESS', 460),
 ('COMICS', 60),
 ('COMMUNICATION', 387),
 ('DATING', 234),
 ('EDUCATION', 156),
 ('ENTERTAINMENT', 149),
 ('EVENTS', 64),
 ('FINANCE', 366),
 ('FOOD_AND_DRINK', 127),
 ('HEALTH_AND_FITNESS', 341),
 ('HOUSE_AND_HOME', 88),
 ('LIBRARIES_AND_DEMO', 85),
 ('LIFESTYLE', 382),
 ('GAME', 1144),
 ('FAMILY', 1972),
 ('MEDICAL', 463),
 ('SOCIAL', 295),
 ('SHOPPING', 260),
 ('PHOTOGRAPHY', 335),
 ('SPORTS', 384),
 ('TRAVEL_AND_LOCAL', 258),
 ('TOOLS', 843),
 ('PERSONALIZATION', 392),
 ('PRODUCTIVITY', 424),
 ('PARENTING', 60),
 ('WEATHER', 82),
 ('VIDEO_PLAYERS', 175),
 ('NEWS_AND_MAGAZINES', 283),
 ('MAPS_AND_NAVIGATION', 137)]

# Comienzo del ejercicio.

Uso take en cada celda para la revisión y sé que no es necesario.

In [ ]:
rdd_reviews = rdd.map(lambda x: (x.Category, (int(x.Reviews), 1)))
rdd_reviews.take(5)

[('ART_AND_DESIGN', (159, 1)),
 ('ART_AND_DESIGN', (967, 1)),
 ('ART_AND_DESIGN', (87510, 1)),
 ('ART_AND_DESIGN', (215644, 1)),
 ('ART_AND_DESIGN', (967, 1))]

In [ ]:
rdd_reviews = rdd_reviews.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
rdd_reviews.collect()

[('ART_AND_DESIGN', (1714440, 65)),
 ('AUTO_AND_VEHICLES', (1163666, 85)),
 ('BEAUTY', (396240, 53)),
 ('BOOKS_AND_REFERENCE', (21959069, 231)),
 ('BUSINESS', (13954552, 460)),
 ('COMICS', (3383276, 60)),
 ('COMMUNICATION', (815462260, 387)),
 ('DATING', (7291278, 234)),
 ('EDUCATION', (39595786, 156)),
 ('ENTERTAINMENT', (59178154, 149)),
 ('EVENTS', (161018, 64)),
 ('FINANCE', (17550728, 366)),
 ('FOOD_AND_DRINK', (8883330, 127)),
 ('HEALTH_AND_FITNESS', (37893743, 341)),
 ('HOUSE_AND_HOME', (3976385, 88)),
 ('LIBRARIES_AND_DEMO', (1037118, 85)),
 ('LIFESTYLE', (12882784, 382)),
 ('GAME', (1585422349, 1144)),
 ('FAMILY', (410226330, 1972)),
 ('MEDICAL', (1585975, 463)),
 ('SOCIAL', (621241422, 295)),
 ('SHOPPING', (115041222, 260)),
 ('PHOTOGRAPHY', (213516650, 335)),
 ('SPORTS', (70830169, 384)),
 ('TRAVEL_AND_LOCAL', (62617919, 258)),
 ('TOOLS', (273185044, 843)),
 ('PERSONALIZATION', (89346140, 392)),
 ('PRODUCTIVITY', (114116975, 424)),
 ('PARENTING', (958331, 60)),
 ('WEATHER', 

In [ ]:
rdd_reviews = rdd_reviews.map(lambda x: (x[0], x[1][0] / x[1][1]))
rdd_reviews.collect()

[('ART_AND_DESIGN', 26376.0),
 ('AUTO_AND_VEHICLES', 13690.188235294117),
 ('BEAUTY', 7476.226415094339),
 ('BOOKS_AND_REFERENCE', 95060.90476190476),
 ('BUSINESS', 30335.982608695653),
 ('COMICS', 56387.933333333334),
 ('COMMUNICATION', 2107137.622739018),
 ('DATING', 31159.30769230769),
 ('EDUCATION', 253819.14102564103),
 ('ENTERTAINMENT', 397168.8187919463),
 ('EVENTS', 2515.90625),
 ('FINANCE', 47952.8087431694),
 ('FOOD_AND_DRINK', 69947.48031496063),
 ('HEALTH_AND_FITNESS', 111125.34604105572),
 ('HOUSE_AND_HOME', 45186.193181818184),
 ('LIBRARIES_AND_DEMO', 12201.388235294118),
 ('LIFESTYLE', 33724.56544502618),
 ('GAME', 1385858.6966783216),
 ('FAMILY', 208025.5223123732),
 ('MEDICAL', 3425.4319654427645),
 ('SOCIAL', 2105903.125423729),
 ('SHOPPING', 442466.23846153845),
 ('PHOTOGRAPHY', 637363.1343283582),
 ('SPORTS', 184453.56510416666),
 ('TRAVEL_AND_LOCAL', 242705.11240310076),
 ('TOOLS', 324062.9228944247),
 ('PERSONALIZATION', 227923.82653061225),
 ('PRODUCTIVITY', 2691

La categoría con mayor cantidad de reviews promedio en sus aplicaciones es:

In [ ]:
rdd_reviews.reduce(lambda x, y: x if x[1] > y[1] else y)

('COMMUNICATION', 2107137.622739018)

Es la categoría con mayor cantidad de reviews promedio porque tiene muchas reviews, pero pocas aplicaciones.